In [4]:
class ChessBoard:

    EMPTY = 0
    PAWN = 1
    KNIGHT = 2
    BISHOP = 3
    ROOK = 4
    QUEEN = 5
    KING = 6

    WHITE = 8
    BLACK = 16

    PIECE_VALUES = {
        PAWN: 1.0,
        KNIGHT: 3.0,
        BISHOP: 3.0,
        ROOK: 5.0,
        QUEEN: 9.0,
        KING: 0.0
    }

    def __init__(self):


        self.board = [
            [self.ROOK | self.BLACK, self.KNIGHT | self.BLACK, self.BISHOP | self.BLACK, self.QUEEN | self.BLACK,
             self.KING | self.BLACK, self.BISHOP | self.BLACK, self.KNIGHT | self.BLACK, self.ROOK | self.BLACK],
            [self.PAWN | self.BLACK] * 8,
            [self.EMPTY] * 8,
            [self.EMPTY] * 8,
            [self.EMPTY] * 8,
            [self.EMPTY] * 8,
            [self.PAWN | self.WHITE] * 8,
            [self.ROOK | self.WHITE, self.KNIGHT | self.WHITE, self.BISHOP | self.WHITE, self.QUEEN | self.WHITE,
             self.KING | self.WHITE, self.BISHOP | self.WHITE, self.KNIGHT | self.WHITE, self.ROOK | self.WHITE]
        ]

        self.turn = self.WHITE  # White to move first
        self.castling_rights = {
            'K': True,  # White kingside
            'Q': True,  # White queenside
            'k': True,  # Black kingside
            'q': True   # Black queenside
        }
        self.en_passant_target = None
        self.halfmove_clock = 0
        self.fullmove_number = 1

    def copy(self):

        new_board = ChessBoard.__new__(ChessBoard)
        new_board.board = [row[:] for row in self.board]
        new_board.turn = self.turn
        new_board.castling_rights = self.castling_rights.copy()
        new_board.en_passant_target = self.en_passant_target
        new_board.halfmove_clock = self.halfmove_clock
        new_board.fullmove_number = self.fullmove_number
        return new_board

    def piece_at(self, row, col):

        if 0 <= row < 8 and 0 <= col < 8:
            return self.board[row][col]
        return None

    def is_occupied(self, row, col):

        return self.piece_at(row, col) != self.EMPTY

    def is_enemy(self, row, col, color):

        piece = self.piece_at(row, col)
        return piece is not None and piece != self.EMPTY and (piece & color) == 0 and (piece & (self.WHITE | self.BLACK)) != 0

    def is_same_color(self, row, col, color):

        piece = self.piece_at(row, col)
        return piece is not None and (piece & color) != 0

    def get_piece_type(self, piece):

        return piece & 7

    def get_piece_color(self, piece):

        return piece & (self.WHITE | self.BLACK)

    def is_check(self, color):

        king_row, king_col = None, None
        for row in range(8):
            for col in range(8):
                piece = self.piece_at(row, col)
                if piece == (self.KING | color):
                    king_row, king_col = row, col
                    break
            if king_row is not None:
                break

        enemy_color = self.BLACK if color == self.WHITE else self.WHITE
        return self.is_square_attacked(king_row, king_col, enemy_color)

    def is_square_attacked(self, row, col, attacking_color):

        pawn_dir = 1 if attacking_color == self.BLACK else -1

        for c_offset in [-1, 1]:
            attack_row = row + pawn_dir
            attack_col = col + c_offset
            if 0 <= attack_row < 8 and 0 <= attack_col < 8:
                piece = self.piece_at(attack_row, attack_col)
                if piece == (self.PAWN | attacking_color):
                    return True

        knight_moves = [(2, 1), (1, 2), (-1, 2), (-2, 1), (-2, -1), (-1, -2), (1, -2), (2, -1)]
        for dr, dc in knight_moves:
            attack_row, attack_col = row + dr, col + dc
            if 0 <= attack_row < 8 and 0 <= attack_col < 8:
                piece = self.piece_at(attack_row, attack_col)
                if piece == (self.KNIGHT | attacking_color):
                    return True

        bishop_dirs = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
        for dr, dc in bishop_dirs:
            r, c = row + dr, col + dc
            while 0 <= r < 8 and 0 <= c < 8:
                piece = self.piece_at(r, c)
                if piece != self.EMPTY:
                    if (piece == (self.BISHOP | attacking_color) or
                        piece == (self.QUEEN | attacking_color)):
                        return True
                    break
                r += dr
                c += dc

        rook_dirs = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        for dr, dc in rook_dirs:
            r, c = row + dr, col + dc
            while 0 <= r < 8 and 0 <= c < 8:
                piece = self.piece_at(r, c)
                if piece != self.EMPTY:
                    if (piece == (self.ROOK | attacking_color) or
                        piece == (self.QUEEN | attacking_color)):
                        return True
                    break
                r += dr
                c += dc

        king_moves = [(1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1)]
        for dr, dc in king_moves:
            attack_row, attack_col = row + dr, col + dc
            if 0 <= attack_row < 8 and 0 <= attack_col < 8:
                piece = self.piece_at(attack_row, attack_col)
                if piece == (self.KING | attacking_color):
                    return True

        return False

    def generate_legal_moves(self):
        moves = []

        for row in range(8):
            for col in range(8):
                piece = self.piece_at(row, col)
                if piece == self.EMPTY or (piece & self.turn) == 0:
                    continue

                piece_type = self.get_piece_type(piece)

                if piece_type == self.PAWN:
                    self._generate_pawn_moves(row, col, moves)

                elif piece_type == self.KNIGHT:
                    self._generate_knight_moves(row, col, moves)

                elif piece_type == self.BISHOP:
                    self._generate_bishop_moves(row, col, moves)


                elif piece_type == self.ROOK:
                    self._generate_rook_moves(row, col, moves)

                elif piece_type == self.QUEEN:
                    self._generate_bishop_moves(row, col, moves)
                    self._generate_rook_moves(row, col, moves)

                elif piece_type == self.KING:
                    self._generate_king_moves(row, col, moves)

        legal_moves = []
        for move in moves:
            temp_board = self.copy()
            temp_board.make_move(move)
            if not temp_board.is_check(self.turn):
                legal_moves.append(move)

        return legal_moves

    def _generate_pawn_moves(self, row, col, moves):

        direction = -1 if self.turn == self.WHITE else 1
        start_row = 6 if self.turn == self.WHITE else 1


        if 0 <= row + direction < 8 and not self.is_occupied(row + direction, col):
            moves.append(((row, col), (row + direction, col)))


            if row == start_row and not self.is_occupied(row + 2 * direction, col):
                moves.append(((row, col), (row + 2 * direction, col)))


        for dc in [-1, 1]:
            if 0 <= col + dc < 8 and 0 <= row + direction < 8:

                if self.is_enemy(row + direction, col + dc, self.turn):
                    moves.append(((row, col), (row + direction, col + dc)))


                if self.en_passant_target == (row + direction, col + dc):
                    moves.append(((row, col), (row + direction, col + dc)))

    def _generate_knight_moves(self, row, col, moves):

        knight_moves = [(2, 1), (1, 2), (-1, 2), (-2, 1), (-2, -1), (-1, -2), (1, -2), (2, -1)]

        for dr, dc in knight_moves:
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < 8 and 0 <= new_col < 8:
                if not self.is_same_color(new_row, new_col, self.turn):
                    moves.append(((row, col), (new_row, new_col)))

    def _generate_bishop_moves(self, row, col, moves):

        bishop_dirs = [(1, 1), (1, -1), (-1, 1), (-1, -1)]

        for dr, dc in bishop_dirs:
            r, c = row + dr, col + dc
            while 0 <= r < 8 and 0 <= c < 8:
                if self.is_same_color(r, c, self.turn):
                    break

                moves.append(((row, col), (r, c)))

                if self.is_enemy(r, c, self.turn):
                    break

                r += dr
                c += dc

    def _generate_rook_moves(self, row, col, moves):

        rook_dirs = [(0, 1), (1, 0), (0, -1), (-1, 0)]

        for dr, dc in rook_dirs:
            r, c = row + dr, col + dc
            while 0 <= r < 8 and 0 <= c < 8:
                if self.is_same_color(r, c, self.turn):
                    break

                moves.append(((row, col), (r, c)))

                if self.is_enemy(r, c, self.turn):
                    break

                r += dr
                c += dc

    def _generate_king_moves(self, row, col, moves):

        king_moves = [(1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1)]

        for dr, dc in king_moves:
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < 8 and 0 <= new_col < 8:
                if not self.is_same_color(new_row, new_col, self.turn):
                    moves.append(((row, col), (new_row, new_col)))


        if self.turn == self.WHITE:

            if self.castling_rights['K'] and not self.is_occupied(7, 5) and not self.is_occupied(7, 6):
                if not self.is_check(self.WHITE) and not self.is_square_attacked(7, 5, self.BLACK):
                    moves.append(((7, 4), (7, 6)))


            if self.castling_rights['Q'] and not self.is_occupied(7, 1) and not self.is_occupied(7, 2) and not self.is_occupied(7, 3):
                if not self.is_check(self.WHITE) and not self.is_square_attacked(7, 3, self.BLACK):
                    moves.append(((7, 4), (7, 2)))
        else:

            if self.castling_rights['k'] and not self.is_occupied(0, 5) and not self.is_occupied(0, 6):
                if not self.is_check(self.BLACK) and not self.is_square_attacked(0, 5, self.WHITE):
                    moves.append(((0, 4), (0, 6)))


            if self.castling_rights['q'] and not self.is_occupied(0, 1) and not self.is_occupied(0, 2) and not self.is_occupied(0, 3):
                if not self.is_check(self.BLACK) and not self.is_square_attacked(0, 3, self.WHITE):
                    moves.append(((0, 4), (0, 2)))

    def make_move(self, move):

        from_pos, to_pos = move
        from_row, from_col = from_pos
        to_row, to_col = to_pos

        piece = self.board[from_row][from_col]
        piece_type = self.get_piece_type(piece)


        self.en_passant_target = None


        if piece_type == self.PAWN and abs(from_row - to_row) == 2:
            self.en_passant_target = (from_row + (to_row - from_row) // 2, from_col)


        if piece_type == self.PAWN and to_col != from_col and self.board[to_row][to_col] == self.EMPTY:

            capture_row = from_row
            capture_col = to_col
            self.board[capture_row][capture_col] = self.EMPTY


        if piece_type == self.KING and abs(from_col - to_col) == 2:

            if to_col > from_col:
                rook_from_col = 7
                rook_to_col = 5

            else:
                rook_from_col = 0
                rook_to_col = 3


            self.board[to_row][rook_to_col] = self.board[to_row][rook_from_col]
            self.board[to_row][rook_from_col] = self.EMPTY


        if piece_type == self.KING:
            if self.turn == self.WHITE:
                self.castling_rights['K'] = False
                self.castling_rights['Q'] = False
            else:
                self.castling_rights['k'] = False
                self.castling_rights['q'] = False


        if piece_type == self.ROOK:
            if from_row == 7 and from_col == 0:
                self.castling_rights['Q'] = False
            elif from_row == 7 and from_col == 7:
                self.castling_rights['K'] = False
            elif from_row == 0 and from_col == 0:
                self.castling_rights['q'] = False
            elif from_row == 0 and from_col == 7:
                self.castling_rights['k'] = False


        if piece_type == self.PAWN or self.board[to_row][to_col] != self.EMPTY:
            self.halfmove_clock = 0
        else:
            self.halfmove_clock += 1


        self.board[to_row][to_col] = self.board[from_row][from_col]
        self.board[from_row][from_col] = self.EMPTY


        self.turn = self.WHITE if self.turn == self.BLACK else self.BLACK


        if self.turn == self.WHITE:
            self.fullmove_number += 1

    def evaluate_position(self):

        score = 0.0


        for row in range(8):
            for col in range(8):
                piece = self.board[row][col]
                if piece == self.EMPTY:
                    continue

                piece_type = self.get_piece_type(piece)
                piece_color = self.get_piece_color(piece)

                if piece_color == self.WHITE:
                    score += self.PIECE_VALUES[piece_type]
                else:
                    score -= self.PIECE_VALUES[piece_type]


        central_squares = [(3, 3), (3, 4), (4, 3), (4, 4)]
        for row, col in central_squares:
            piece = self.board[row][col]
            if piece == self.EMPTY:
                continue

            piece_type = self.get_piece_type(piece)
            piece_color = self.get_piece_color(piece)

            if piece_type in [self.PAWN, self.KNIGHT]:
                if piece_color == self.WHITE:
                    score += 0.3
                else:
                    score -= 0.3

        current_turn = self.turn


        self.turn = self.WHITE
        white_mobility = len(self.generate_legal_moves())

        self.turn = self.BLACK
        black_mobility = len(self.generate_legal_moves())


        self.turn = current_turn

        score += 0.1 * (white_mobility - black_mobility)


        if self.turn == self.BLACK:
            score = -score

        return score

    def get_algebraic_notation(self, move):

        from_pos, to_pos = move
        from_row, from_col = from_pos
        to_row, to_col = to_pos

        files = 'abcdefgh'
        ranks = '87654321'

        from_square = files[from_col] + ranks[from_row]
        to_square = files[to_col] + ranks[to_row]

        return from_square + to_square

    def __str__(self):

        result = []
        for row in range(8):
            row_str = []
            for col in range(8):
                piece = self.board[row][col]
                if piece == self.EMPTY:
                    row_str.append('.')
                else:
                    piece_type = self.get_piece_type(piece)
                    piece_color = self.get_piece_color(piece)

                    if piece_color == self.WHITE:
                        if piece_type == self.PAWN:
                            row_str.append('P')
                        elif piece_type == self.KNIGHT:
                            row_str.append('N')
                        elif piece_type == self.BISHOP:
                            row_str.append('B')
                        elif piece_type == self.ROOK:
                            row_str.append('R')
                        elif piece_type == self.QUEEN:
                            row_str.append('Q')
                        elif piece_type == self.KING:
                            row_str.append('K')
                    else:
                        if piece_type == self.PAWN:
                            row_str.append('p')
                        elif piece_type == self.KNIGHT:
                            row_str.append('n')
                        elif piece_type == self.BISHOP:
                            row_str.append('b')
                        elif piece_type == self.ROOK:
                            row_str.append('r')
                        elif piece_type == self.QUEEN:
                            row_str.append('q')
                        elif piece_type == self.KING:
                            row_str.append('k')

            result.append(' '.join(row_str))

        return '\n'.join(result) + '\n' + ('White' if self.turn == self.WHITE else 'Black') + ' to move'


def beam_search_chess(board, beam_width, depth_limit):


    class BeamNode:
        def __init__(self, board, move_sequence, score):
            self.board = board.copy()
            self.move_sequence = move_sequence.copy()
            self.score = score


    current_beam = [BeamNode(board, [], board.evaluate_position())]


    for depth in range(depth_limit):
        next_beam = []

        for node in current_beam:

            legal_moves = node.board.generate_legal_moves()


            if not legal_moves:
                continue


            move_evaluations = []
            for move in legal_moves:
                new_board = node.board.copy()
                new_board.make_move(move)

                new_sequence = node.move_sequence + [move]
                score = new_board.evaluate_position()

                move_evaluations.append((move, new_board, new_sequence, score))


            is_white_turn = node.board.turn == board.WHITE
            move_evaluations.sort(key=lambda x: x[3], reverse=is_white_turn)


            for move, new_board, new_sequence, score in move_evaluations[:beam_width]:
                next_beam.append(BeamNode(new_board, new_sequence, score))


        is_white_to_move = board.turn == board.WHITE
        next_beam.sort(key=lambda node: node.score, reverse=is_white_to_move)
        current_beam = next_beam[:beam_width]


        if not current_beam:
            break


    if current_beam:
        is_white_to_move = board.turn == board.WHITE
        best_node = max(current_beam, key=lambda node: node.score) if is_white_to_move else min(current_beam, key=lambda node: node.score)
        return best_node.move_sequence, best_node.score
    else:
        return [], 0.0


def format_move_sequence(board, move_sequence):

    formatted_moves = []
    temp_board = board.copy()

    for move in move_sequence:
        formatted_moves.append(temp_board.get_algebraic_notation(move))
        temp_board.make_move(move)

    return formatted_moves


def main():

    board = ChessBoard()
    beam_width = 3
    depth_limit = 4

    move_sequence, score = beam_search_chess(board, beam_width, depth_limit)

    print(f"Board state:\n{board}")
    print(f"Best move sequence: {format_move_sequence(board, move_sequence)}")
    print(f"Evaluation score: {score}")


if __name__ == "__main__":
    main()

Board state:
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
White to move
Best move sequence: ['a2a3', 'e7e6', 'f2f3', 'f8a3']
Evaluation score: -2.2


In [11]:
#Task 2
import random
import math

def distance(a, b):
    return math.hypot(a[0] - b[0], a[1] - b[1])

def total_distance(route):
    return sum(distance(route[i], route[(i + 1) % len(route)]) for i in range(len(route)))

def get_neighbor(route):
    i, j = sorted(random.sample(range(len(route)), 2))
    neighbor = route[:]
    neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
    return neighbor

def hill_climb(locations, max_iterations=10000):
    current = locations[:]
    random.shuffle(current)
    current_distance = total_distance(current)
    for _ in range(max_iterations):
        neighbor = get_neighbor(current)
        neighbor_distance = total_distance(neighbor)
        if neighbor_distance < current_distance:
            current = neighbor
            current_distance = neighbor_distance
    return current, current_distance

locations = [(random.uniform(0, 100), random.uniform(0, 100)) for _ in range(10)]
route, dist = hill_climb(locations)
print("Optimized route:", route)
print("Total distance:", dist)


Optimized route: [(19.574799762037355, 23.068337464318844), (9.80974976600434, 80.70040243985542), (16.536939819479922, 83.44791828145914), (47.807040113563126, 84.74974736525259), (63.09097793052278, 73.14597130938198), (75.30291111496975, 56.814391097349514), (67.83691873978196, 45.80670972503421), (78.1578161855677, 10.912028908305139), (43.49474381692541, 27.407792549823974), (39.98741009302292, 16.190617081157434)]
Total distance: 257.9701302383297


In [7]:
#Task 3
import random
import numpy as np

cities = [(random.uniform(0, 100), random.uniform(0, 100)) for _ in range(10)]

def distance(a, b):
    return np.linalg.norm(np.array(a) - np.array(b))

def total_distance(route):
    return sum(distance(cities[route[i]], cities[route[(i + 1) % len(route)]]) for i in range(len(route)))

def create_route():
    route = list(range(len(cities)))
    random.shuffle(route)
    return route

def initial_population(size):
    return [create_route() for _ in range(size)]

def rank_routes(population):
    return sorted(population, key=total_distance)

def selection(population, elite_size):
    ranked = rank_routes(population)
    return ranked[:elite_size] + random.choices(ranked, k=len(ranked) - elite_size)

def crossover(parent1, parent2):
    start, end = sorted(random.sample(range(len(parent1)), 2))
    child = [None]*len(parent1)
    child[start:end] = parent1[start:end]
    ptr = end
    for i in range(len(parent2)):
        if parent2[i] not in child:
            if ptr >= len(child):
                ptr = 0
            child[ptr] = parent2[i]
            ptr += 1
    return child

def mutate(route, mutation_rate):
    for i in range(len(route)):
        if random.random() < mutation_rate:
            j = random.randint(0, len(route) - 1)
            route[i], route[j] = route[j], route[i]
    return route

def evolve(population, elite_size, mutation_rate):
    selected = selection(population, elite_size)
    children = [crossover(selected[i], selected[(i + 1) % len(selected)]) for i in range(len(population))]
    mutated = [mutate(child, mutation_rate) for child in children]
    return mutated

population_size = 100
elite_size = 20
mutation_rate = 0.01
generations = 500

population = initial_population(population_size)

for i in range(generations):
    population = evolve(population, elite_size, mutation_rate)

best_route = rank_routes(population)[0]
print("Best Route:", best_route)
print("Distance:", total_distance(best_route))


Best Route: [0, 4, 7, 9, 1, 8, 6, 3, 2, 5]
Distance: 328.2835588782905
